In [2]:
#import library
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import string
import ast
from tensorflow.keras.models import load_model

nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('stopwords')
nltk.download('tokenizers')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hanifah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hanifah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hanifah\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hanifah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index


False

In [3]:
from tensorflow.keras.models import load_model
model_dnn = load_model('model_lstm')

In [4]:
#open chatwords.txt
with open('chatwords.txt') as f:
    data = f.read()
chatwords =  ast.literal_eval(data)
chatwords

#open abbreviation.txt
with open('abbreviation.txt') as ab:
    ab2 = ab.read()
abbreviation =  ast.literal_eval(ab2)
abbreviation

#define stopwords
stop_words = stopwords.words('english')

#define lemmatizer
lem = WordNetLemmatizer()
def check_chatwords(text):
    temp=[]
    for chat in text.split():
        if chat.upper() in chatwords:
            temp.append(chatwords[chat.upper()])
        else:
            temp.append(chat)
    return " ".join(temp)

In [5]:
def lower(text):
    data = text.lower()
    return data 

def check_abbr(text):
    temp2=[]
    for abbr in text.split():
      if abbr in abbreviation:
          temp2.append(abbreviation[abbr])
      else:
          temp2.append(abbr)

    return " ".join(temp2)

def check_punctuation(text):
    data = re.sub("[^a-zA-Z]",' ', text)
    data = re.sub('[[^]]*]', ' ', data)
    data = re.sub(r"\n", " ", data)
    data = data.strip()
    data = ' '.join(data.split())
    return data

def token_stopwords_lemma(text):
    tokens = word_tokenize(text)
    stop_words2 = ' '.join([word for word in tokens if word not in stop_words])
    data = [lem.lemmatize(word) for word in stop_words2.split()]
    data = ' '.join(data)
    return data

In [11]:
inf = ["My first time trying this and fall in love instantly! I use it as primer. Makes my makeup look smooth and stays the whole day","It wasn’t a great base for my foundation. My foundation slid all over my face while applying. It rubbed off time i started setting my face! It wasn’t tacky at all. But it was moisturizing"]

In [12]:
# convert to dataframe
df1= pd.DataFrame()
df1['Review_Text'] = inf

In [13]:
df1['Review_Text'] = df1['Review_Text'].apply(lambda j: check_chatwords(j))
df1['Review_Text'] = df1['Review_Text'].apply(lambda k: lower(k))
df1['Review_Text'] = df1['Review_Text'].apply(lambda l: check_abbr(l))
df1['Review_Text'] = df1['Review_Text'].apply(lambda m: check_punctuation(m))
df1['Review_Text'] = df1['Review_Text'].apply(lambda n: token_stopwords_lemma(n))

C:\Users\Hanifah\AppData\Local\Temp\ipykernel_6928\714082318.py:17: FutureWarning: Possible nested set at position 1
  data = re.sub('[[^]]*]', ' ', data)


In [14]:
model = model_dnn.predict(df1[['Review_Text']])
model=np.where(model >=0.5, 1, 0)
model

1/1 [==============================] - 1s 1s/step


array([[0],
       [0]])

In [15]:
df_pred_model = pd.DataFrame(model, columns=['Pred_review']) 
df_pred_model

,Pred_review
0,0
1,0


In [16]:
df_combined = pd.concat([df1, df_pred_model], axis=1)
df_combined

,Review_Text,Pred_review
0,first time trying fall love instantly use prim...,0
1,great base foundation foundation slid face app...,0
